In [49]:
import re
from collections import Counter
import math

from nltk import ngrams
import numpy as np
import random

In [50]:
RUS_ALPHABET = 'абвгдежзийклмнопрстуфхцчшщъыьэюя '
ENCRYPTION_KEY = 'ж зьсйхцэливяюбучмоыдещтгпфъашнкр'

# Основные данные

#### Текст

In [51]:
def text_preprocessing(text: str) -> str:
        return re.sub('[^а-я]+', ' ',  text.lower())

assert text_preprocessing('ЗАГЛАВНЫЕ, строчные. Пунктуация! Цифры 123?') == 'заглавные строчные пунктуация цифры '

In [52]:
corpus = ['WarAndPeace.txt', 'AnnaKarenina.txt'] # 'WarAndPeaceEng.txt', 
text = ""
for file_name in corpus:
    with open(f'corpora/{file_name}') as f:
        text += f.read()

In [53]:
text = text_preprocessing(text)
text[:1000]

' война и мир самый известный роман льва николаевича толстого как никакое другое произведение писателя отражает глубину его мироощущения и философии эта книга из разряда вечных потому что она обо всем о жизни и смерти о любви и чести о мужестве и героизме о славе и подвиге о войне и мире первый том знакомит с высшим обществом россии века показаны взаимоотношения между родителями и детьми в семье ростовых сватовство у болконских интриги у безуховых вечера в салоне фрейлины а п шерер балы в москве и петербурге лев николаевич толстойчасть первая часть орая часть третья лев николаевич толстой война и мир том часть первая е поместья мой верный раб ну что князь генуа и лукка стали не больше как поместьями фамилии бонапарте нет я вас предупреждаю если вы мне не скажете что у нас война если вы еще позволите себе защищать все гадости все ужасы этого антихриста право я верю что он антихрист я вас больше не знаю вы уж не друг мой вы уж не мой верный раб как вы говорите ну здравствуйте здравствуйт

#### Исходное сообщение для шифрации

In [54]:
message = "\
Кутузов чрез своего лазутчика получил 1-го ноября известие, ставившее командуемую им армию почти в безвыходное положение. \
Лазутчик доносил, что французы в огромных силах, перейдя венский мост, направились на путь сообщения Кутузова с войсками, шедшими из России.\
Ежели бы Кутузов решился оставаться в Кремсе, то полуторастатысячная армия Наполеона отрезала бы его от всех сообщений, окружила бы его сорокатысячную изнуренную армию, и он находился бы в положении Мака под Ульмом.\
Ежели бы Кутузов решился оставить дорогу, ведшую на сообщения с войсками из России, то он должен был вступить без дороги в неизвестные края Богемских гор, защищаясь от превосходного силами неприятеля, и оставить всякую надежду на сообщение с Буксгевденом."
message = text_preprocessing(message)
message

'кутузов чрез своего лазутчика получил го ноября известие ставившее командуемую им армию почти в безвыходное положение лазутчик доносил что французы в огромных силах перейдя венский мост направились на путь сообщения кутузова с войсками шедшими из россии ежели бы кутузов решился оставаться в кремсе то полуторастатысячная армия наполеона отрезала бы его от всех сообщений окружила бы его сорокатысячную изнуренную армию и он находился бы в положении мака под ульмом ежели бы кутузов решился оставить дорогу ведшую на сообщения с войсками из россии то он должен был вступить без дороги в неизвестные края богемских гор защищаясь от превосходного силами неприятеля и оставить всякую надежду на сообщение с буксгевденом '

#### Шифрация

In [55]:
def create_cipher_dict(cipher):
    cipher_dict = {}
    alphabet_list = list(RUS_ALPHABET)
    for i in range(len(cipher)):
        cipher_dict[alphabet_list[i]] = cipher[i]
    return cipher_dict

def apply_cipher_on_text(text, cipher):
    cipher_dict = create_cipher_dict(cipher) 
    text = list(text)
    newtext = ""
    for elem in text:
        newtext += cipher_dict[elem.lower()]
    return newtext

In [56]:
message_encrypted = apply_cipher_on_text(message, ENCRYPTION_KEY)
message_encrypted

'иыоыцбзртчйцрмзбйьбрвжцыотэижрубвытэврьбрюбк чкрэцзймоэйрможзэзгййрибяжюсыйяынрэяржчяэнрубтоэрзр йцзъебсюбйрубвбхйюэйрвжцыотэирсбюбмэвртобрдчжющыцързрбьчбяюъермэвжеруйчйлскрзйюмиэлрябморюжучжзэвэмарюжруыоармбб пйюэкриыоыцбзжрмрзблмижяэргйсгэяэрэцрчбммээрйхйвэр ъриыоыцбзрчйгэвмкрбможзжоамкрзричйямйробрубвыобчжможоъмктюжкржчяэкрюжубвйбюжрбочйцжвжр ърйьбрборзмйермбб пйюэлрбичыхэвжр ърйьбрмбчбижоъмктюынрэцюычйююынржчяэнрэрбюрюжебсэвмкр ързрубвбхйюээряжижрубсрываябярйхйвэр ъриыоыцбзрчйгэвмкрбможзэоарсбчбьырзйсгынрюжрмбб пйюэкрмрзблмижяэрэцрчбммээробрбюрсбвхйюр ъврзмоыуэоар йцрсбчбьэрзрюйэцзймоюъйричжкр бьйямиэерьбчрцжпэпжкмарборучйзбмебсюбьбрмэвжяэрюйучэкойвкрэрбможзэоарзмкиынрюжсйхсырюжрмбб пйюэйрмр ыимьйзсйюбяр'

In [57]:
# оценка качества дешифрации
def score(message: str, message_decrypted: str):
    assert len(message) == len(message_decrypted)
    quality = [message[i] == message_decrypted[i] for i in range(len(message))]
    return sum(quality) / len(message)

assert score("абвг", "аxxг" ) == 0.5

# Частоты букв в текстах Л. Н. Толстого

In [58]:
def freq_counter(text: str) -> Counter:
    letter_counter = Counter()
    letter_counter.update(text)
    total_letters = sum(letter_counter.values())
    for key in letter_counter.keys():
        letter_counter[key] = letter_counter[key] / total_letters
    return letter_counter

In [59]:
assert freq_counter('aaaбв') == Counter({'a': 0.6, 'б': 0.2, 'в': 0.2})

In [60]:
letter_counter = freq_counter(text)

In [61]:
letter_counter.most_common(5)

[(' ', 0.16594214613758704),
 ('о', 0.09567220877824625),
 ('е', 0.07107016044665365),
 ('а', 0.06942095669214891),
 ('н', 0.056994189294033014)]

# Базовый частотный метод по Шерлоку Холмсу

In [62]:
encrypted_letter_counter = freq_counter(message_encrypted)
encrypted_letter_counter.most_common(5)

[('р', 0.14783821478382148),
 ('б', 0.09623430962343096),
 ('э', 0.07252440725244072),
 ('й', 0.06694560669456066),
 ('м', 0.058577405857740586)]

In [63]:
# нахожу близкие по частоте буквы в справочном тексте и в зашифрованном сообщение -> создаю словарь декодирования
def find_mapping_by_freq(counter_source: Counter, counter_encrypted: Counter) -> dict:
    
    mapping = {}
    
    for letter in counter_encrypted:
        
        # find position of encrypted letter
        counter_source['find_position'] = counter_encrypted[letter]
        sorted_by_frequency_lst = counter_source.most_common()
        index = sorted_by_frequency_lst.index(('find_position', counter_encrypted[letter])) # index in sorted list

        # find closest decrypted letter
        dist_befor = abs(sorted_by_frequency_lst[index - 1][1] - sorted_by_frequency_lst[index][1])
        dist_after = abs(sorted_by_frequency_lst[index + 1][1] - sorted_by_frequency_lst[index][1])
        if dist_befor <= dist_after:
            mapping[letter] = sorted_by_frequency_lst[index - 1][0]
        else:
            mapping[letter] = sorted_by_frequency_lst[index + 1][0]
        
        # keep original state
        del letter_counter['find_position']
            
    return mapping

assert find_mapping_by_freq(Counter({"a": 0.1, "б": 0.2, "в": 0.3}), 
                            Counter({"г": 0.14, "д": 0.26})) == {'г': 'a', 'д': 'в'}

In [64]:
decipher_mapping = find_mapping_by_freq(letter_counter, encrypted_letter_counter)

In [65]:
message_decrypted = "".join([decipher_mapping[letter] for letter in message_encrypted])
message_decrypted

'двсвмов жрам нвоаыо римвсжеди яорвжер ыо тодпрд емвансеа нсивевйаа додитпвадвж ед ирдеж яожсе в памвьжоптоа яорожатеа римвсжед потонер жсо фритфвмь в оыродтьж нериж яарахпд ватндех донс тияриверенж ти явсж ноопйатед двсвмови н вохндиде йапйеде ем роннее ажаре пь двсвмов райернд онсивисжнд в драдна со яорвсоринсисьнджтид ирдед тияораоти осрамири пь аыо ос внаж ноопйатех одрвжери пь аыо нородисьнджтвж емтвраттвж ирдеж е от тижопернд пь в яорожатее диди яоп врждод ажаре пь двсвмов райернд онсивесж пороыв вапйвж ти ноопйатед н вохндиде ем роннее со от поржат пьр внсвяесж пам пороые в таемванстьа дрид поыадндеж ыор мийейиднж ос яравонжоптоыо нериде таяредсард е онсивесж внддвж типажпв ти ноопйатеа н пвдныавпатод '

In [66]:
score(message, message_decrypted)

0.3291492329149233

# Частоты биграмм в текстах Л. Н. Толстого

In [67]:
def freq_counter_bigrams(text_tokenized: str) -> Counter:
    bigram_lst = list(ngrams(text_tokenized, n=2)) # list of pairs like ('h', 'e')
    counter_bi = Counter([bi_tup[0] + bi_tup[1] for bi_tup in bigram_lst])
    total = sum(counter_bi.values())
    for key in counter_bi.keys():
        counter_bi[key] = counter_bi[key] / total
    return counter_bi

assert freq_counter_bigrams('hey hello') == Counter({'he': 0.25,
                                                      'ey': 0.125,
                                                      'y ': 0.125,
                                                      ' h': 0.125,
                                                      'el': 0.125,
                                                      'll': 0.125,
                                                      'lo': 0.125})

In [68]:
counter_bi = freq_counter_bigrams(text)

In [69]:
counter_bi.most_common(5)

[('о ', 0.023195787001138106),
 ('а ', 0.01809334685430074),
 ('и ', 0.017992410066819297),
 ('е ', 0.017961615792672418),
 (' с', 0.01591764584617319)]

# Частотный метод на биграммах

In [70]:
encrypted_counter_bi = freq_counter_bigrams(message_encrypted)

In [71]:
encrypted_counter_bi.most_common(5)

[('эр', 0.018156424581005588),
 ('рм', 0.01675977653631285),
 ('кр', 0.01675977653631285),
 ('рз', 0.01675977653631285),
 ('рю', 0.013966480446927373)]

#### Дешифрация

In [72]:
decipher_mapping_bi = find_mapping_by_freq(counter_bi, encrypted_counter_bi)

In [73]:
message_decrypted = "".join([decipher_mapping_bi[message_encrypted[i: i+2]] for i in range(0, len(message_encrypted)-1, 2)]) + " "
message_decrypted

'вово мя их м с макя го э э э о к эакакготоихих сво м оак сво михихго м э эакво какак к к к эа я  эак э мво к к эвого кго э э э м мя  о э мих михвогоя вогоих э с о э оихихих сто м э эя ак оакво оихго о оя гово м мго свово м э с с э мака  э э эа во мя воа акака гогоя во м м э о кто овоаких ся акак э эя  к э м м оак э э о сак к с о ких м оакя  эготогоакя ак с э мво м мго э эихихготогоакя вогово э к эво кихихтоак как к ка  мто э м о ктогоя  к эвогоа  эво о михих эакакака гогоя во м м э о кто овоакго мгоих с эих к о с м мто м с с э мака во мя воа  мтоак михтотоих с оихакго э м мгоих стоихак эихихго м сих э э э мгоих эихихихгоак оя  мих мвого с оака  эак михих кто овоакгоакихвотоихак это ово м мго какихих эихто м '

In [74]:
score(message, message_decrypted)

0.12412831241283125

# Markov Chain Monte Carlo

<br> Decryptor score:
<img src="Score.png">
<br> R( β₁,β₂) record the number of times that specific pair(e.g. “TH”) appears consecutively in the reference text.
<br> Fₓ(β₁,β₂) record the number of times that pair appears when the ciphertext is decrypted using the decryption key x.

In [75]:
# метрика качества дешифратора
def get_cipher_score(text, cipher, scoring_params):
    cipher_dict = create_cipher_dict(cipher)
    decrypted_text = apply_cipher_on_text(text, cipher)
    scored_f = freq_counter_bigrams(decrypted_text)
    cipher_score = 0
    for k,v in scored_f.items():
        if k in scoring_params:
            cipher_score += v * math.log(scoring_params[k])
    return cipher_score

# счетчик биграм
def freq_counter_bigrams(text: str) -> Counter:
    text = text.lower()
    bigram_lst = list(ngrams(text, n=2)) # list of pairs like [('h', 'e'), ('t', 'h'), ... ]
    counter_bi = Counter([bi_tup[0] + bi_tup[1] for bi_tup in bigram_lst]) 
    return counter_bi

# монетка, выдающая 1 с вероятностью p
def random_coin(p):
    unif = random.uniform(0,1)
    if unif>=p:
        return False
    else:
        return True

# случайная перестановка 2х элементов в дешифраторе
def generate_cipher(cipher):
    pos1 = random.randint(0, len(list(cipher))-1)
    pos2 = random.randint(0, len(list(cipher))-1)
    if pos1 == pos2:
        return generate_cipher(cipher)
    else:
        cipher = list(cipher)
        cipher[pos1], cipher[pos2] = cipher[pos2], cipher[pos1]
        return "".join(cipher)

#  Markov Chain Monte Carlo
def MCMC_decrypt(n_iter, cipher_text, scoring_params, initial_cipher):
    current_cipher = initial_cipher
    state_keeper = set()
    best_state = ''
    score = 0
    for i in range(n_iter):
        state_keeper.add(current_cipher)
        proposed_cipher = generate_cipher(current_cipher)
        score_current_cipher = get_cipher_score(cipher_text, current_cipher, scoring_params)
        score_proposed_cipher = get_cipher_score(cipher_text, proposed_cipher, scoring_params)
        acceptance_probability = min(1, math.exp(score_proposed_cipher-score_current_cipher))
        if score_current_cipher > score:
            best_state = current_cipher
        if random_coin(acceptance_probability):
            current_cipher = proposed_cipher
        if i % 500 == 0:
            print("iter", i, ":", apply_cipher_on_text(cipher_text, current_cipher)[0:99])
    return state_keeper,best_state

In [76]:
# reference text
scoring_params = freq_counter_bigrams(text)
print("\n")

# secret message
print("Secret message:", message)
message_encrypted = apply_cipher_on_text(message, ENCRYPTION_KEY)
print("Encrypted message:", message_encrypted)
print("\n")

# decryption
states, best_state = MCMC_decrypt(10000, message_encrypted, scoring_params, RUS_ALPHABET)
print("\n")
print("Decoded Text:", apply_cipher_on_text(message_encrypted, best_state))
print("\n")



Secret message: кутузов чрез своего лазутчика получил го ноября известие ставившее командуемую им армию почти в безвыходное положение лазутчик доносил что французы в огромных силах перейдя венский мост направились на путь сообщения кутузова с войсками шедшими из россии ежели бы кутузов решился оставаться в кремсе то полуторастатысячная армия наполеона отрезала бы его от всех сообщений окружила бы его сорокатысячную изнуренную армию и он находился бы в положении мака под ульмом ежели бы кутузов решился оставить дорогу ведшую на сообщения с войсками из россии то он должен был вступить без дороги в неизвестные края богемских гор защищаясь от превосходного силами неприятеля и оставить всякую надежду на сообщение с буксгевденом 
Encrypted message: иыоыцбзртчйцрмзбйьбрвжцыотэижрубвытэврьбрюбк чкрэцзймоэйрможзэзгййрибяжюсыйяынрэяржчяэнрубтоэрзр йцзъебсюбйрубвбхйюэйрвжцыотэирсбюбмэвртобрдчжющыцързрбьчбяюъермэвжеруйчйлскрзйюмиэлрябморюжучжзэвэмарюжруыоармбб пйюэкриыоыцбзжрмрзблмижяэргйсгэяэрэ

# Расшифровка сообщения

In [ ]:
message_encrypted = "←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸ ↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊ ↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒ ↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝← ⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏"
NEW_ALPHABET = '⇊⇴⇏↟↳⇠↹↤⇷⇛⇸←↝↷↞↘↙⇆↲↾⇒⇌⇯ ⇞⇰⇈↨⇙' + 'abcd' # additional symbols to match lenght 33 of RUS_ALPHABET

In [ ]:
def create_cipher_dict(cipher):
    cipher_dict = {}
    alphabet_list = list(NEW_ALPHABET)
    for i in range(len(cipher)):
        cipher_dict[alphabet_list[i]] = cipher[i]
    return cipher_dict

In [47]:
states, best_state = MCMC_decrypt(10000, message_encrypted, scoring_params, RUS_ALPHABET)
print("\n")
print("Decoded Text:", apply_cipher_on_text(message_encrypted, best_state))
print("\n")

iter 0 : лефгжнажнгзгылжмйцдбфъмаужгфгжийшыгжмйцдбфъмаужылхеыжьжкчыйрйжеййтслмгожхйыйцаужфлрхйжицйшгыбыъжехй
iter 500 : еоду пя путусе винзадцвяй уду миксу винзадцвяй серос ь хлсиги оиижшевуч рисиняй дегри мникусасц ори
iter 1000 : еули пя пикиве носчалыняй или бодви носчалыняй верув ь хтвого уоожшению ровосяй легро бсодивавы уро
iter 1500 : ерни пя пимиве лосканыляй ини бошви лосканыляй ветрв ь чувого роожделию товосяй негто бсошивавы рто
iter 2000 : ерни пя пимиве лосканыляй ини ботви лосканыляй ведрв ь чувого роожцелию довосяй негдо бсотивавы рдо
iter 2500 : ерли пы пижите носкаляный или вобти носкаляный тедрт ь чутого роомщению дотосый легдо всобитатя рдо
iter 3000 : ерли ды димите носкальный или вожти носкальный тепрт я шутого рообчению потосый легпо всожитать рпо
iter 3500 : ерти бы бимиле носкатьный ити водли носкатьный лепрл я цулого роожчению полосый тегпо всодилаль рпо
iter 4000 : ерти бы бимиле носкатьный ити водли носкатьный лепрл я цулого роожчению полосый тегпо всодилаль 